### Implementation of the brain-score/language pipeline on the German Emotive Idioms dataset

In [1]:
# %load_ext autoreload
# %autoreload 2

import os
import sys
import json

sys.path.append("brain-score-language")

import torch
import random 
import argparse
import warnings
import numpy as np
import pandas as pd
import pickle as pkl

from brainio.assemblies import merge_data_arrays
from brainscore_language import ArtificialSubject, load_benchmark, score
from brainscore_language.benchmarks.german_emotive_idioms import GermanEmotiveIdioms
from brainscore_language.model_helpers.huggingface import HuggingfaceSubject
from matplotlib import pyplot as plt
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, AutoModelForSeq2SeqLM

/Users/emilyzhou/Desktop/PSYC626/CS-626-Project/.venv/lib/python3.11/site-packages/brainscore_core/metrics/__init__.py:16: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [2]:
from huggingface_hub import login

from huggingface_hub import notebook_login
notebook_login()

### Load and visualize model

In [3]:
model = HuggingfaceSubject(model_id='distilgpt2', region_layer_mapping={})
# model = HuggingfaceSubject(model_id='meta-llama/Llama-2-7b', region_layer_mapping={})
print(model.basemodel)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


### Load German Idioms benchmark dataset and score GPT-2 layers

In [4]:
# Load neural data
# TODO: Replace placeholder data with actual fMRI activations

from brainscore_core.metrics import Score

neural_data_folder = os.path.join("..", "emotive_idioms_dataset", "processed_data")
neural_data_file = os.path.join(neural_data_folder, "dummy_data.npy")
ceilings_file = os.path.join(neural_data_folder, "dummy_ceilings.json")
neural_data = np.load(neural_data_file)
with open(ceilings_file, "r") as f:
    ceiling = json.load(f)

print(f"Neural data: {neural_data.shape}")
print(f"Ceilings: {list(ceiling.keys())}")
# print(ceiling)

# Cast ceiling to Score
score = ceiling['score']
raw = ceiling['raw']
ceiling = Score(score)
ceiling.raw = raw
ceiling.name = 'data'
print(ceiling)

Neural data: (12, 150)
Ceilings: ['score', 'raw', 'neuroid_id']
<xarray.Score 'data' ()>
array(0.06087531)


/var/folders/nr/k2l4pvsd4vqc6pt6xf46_mqm0000gn/T/ipykernel_66371/2369733596.py:21: FutureWarning: Setting attribute 'raw' on a 'Score' object. Explicitly define __slots__ to suppress this warning for legitimate custom attributes and raise an error when attempting variables assignments.
  ceiling.raw = raw


In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

benchmark = load_benchmark('Pereira2018.243sentences-linear')
data = benchmark.data
df = benchmark.data.to_dataframe()
# print(data)
# print(data['presentation']['stimulus_id'])
# print(data['presentation']['stimulus'])
# print(data['presentation']['sentence'])

# layer_scores = []
# for layer in tqdm([f'transformer.h.{block}.ln_1' for block in range(6)], desc='layers'):
#     layer_model = HuggingfaceSubject(model_id='distilgpt2', region_layer_mapping={
#         ArtificialSubject.RecordingTarget.language_system: layer})
#     layer_score = benchmark(layer_model)
#     # package for xarray
#     layer_score = layer_score.expand_dims('layer')
#     layer_score['layer'] = [layer]
#     layer_scores.append(layer_score)
# layer_scores = merge_data_arrays(layer_scores)
# print(layer_scores)

In [6]:
benchmark = GermanEmotiveIdioms(neural_data, ceiling)
data = benchmark.data
df = benchmark.data.to_dataframe()
# print(data)
# print(data['presentation']['stimulus_id'])

layer_scores = []
for layer in tqdm([f'transformer.h.{block}.ln_1' for block in range(6)], desc='layers'):
    layer_model = HuggingfaceSubject(model_id='distilgpt2', region_layer_mapping={
        ArtificialSubject.RecordingTarget.language_system: layer})
    layer_score = benchmark(layer_model)
    # package for xarray
    layer_score = layer_score.expand_dims('layer')
    layer_score['layer'] = [layer]
    layer_scores.append(layer_score)
layer_scores = merge_data_arrays(layer_scores)
print(layer_scores)

layers: 100%|██████████| 6/6 [00:22<00:00,  3.71s/it]

<xarray.Score (layer: 6)>
array([1., 0., 1., 1., 1., 0.])
Coordinates:
  * layer    (layer) <U20 'transformer.h.0.ln_1' ... 'transformer.h.5.ln_1'
Attributes:
    raw:                          <xarray.Score (layer: 1)>\narray([0.2])\nCo...
    ceiling:                      <xarray.Score 'data' ()>\narray(0.06087531)
    original_out_of_range_score:  3.2854040966333793
